Przygotowanie danych do modeli predykcyjnych jest fundamentalne dla sukcesu w dziedzinie uczenia maszynowego i statystyki. Kluczowe cele i zakres tej pracy obejmują:

1. Poprawa dokładności modelu:
Dane nieprzygotowane lub nieprzetworzone często zawierają szum, braki oraz błędy, które mogą wprowadzić w błąd algorytmy uczenia maszynowego, prowadząc do niewłaściwych prognoz. Poprzez wstępne przetwarzanie danych, takie jak czyszczenie i normalizacja, możemy zwiększyć dokładność przewidywań modelu.

2. Zapewnienie zgodności danych:
Wielkość i format danych wejściowych muszą być spójne w całym zestawie danych. Przygotowanie danych zapewnia, że wszystkie zmienne są w odpowiednich formatach i zakresach, co umożliwia algorytmom efektywne i poprawne przetwarzanie informacji.

3. Obsługa danych kategorialnych:
Modele predykcyjne, szczególnie te oparte na matematyce, rozumieją dane głównie w formie liczbowej. Przekształcenie danych kategorialnych (takich jak płeć czy kolor) na liczbowe poprzez kodowanie jest kluczowe dla umożliwienia modelom przetwarzania tych informacji.

4. Uzupełnianie brakujących danych:
Brak danych może wprowadzić znaczne zakłócenia w analizie i skuteczności modeli. Wypełnianie luk danych przez tzw. imputację (czyli np. zastąpienie braku danych wzrostu ucznia średnim wzrostem wszystkich uczniów) lub usuwanie rekordów z brakującymi wartościami pomaga zapewnić ciągłość i kompletność zestawu danych.


5. Wykrywanie i usuwanie obserwacji odstających (outlierów):
Dane odstające to takie, które odbiegają wartością liczbową od większości innych obserwacji. Np. wzrost człowieka 2m 89 cm.
Dane odstające mogą skrzywić wyniki modeli i prowadzić do nieprawidłowych generalizacji. Ich identyfikacja i ewentualne usunięcie lub korekta są kluczowe dla utrzymania integralności modeli predykcyjnych.

Podsumowując:
Przygotowanie danych jest krytycznym etapem w procesie budowania modeli predykcyjnych. Jego celem jest przekształcenie surowych danych w czysty, kompletny i odpowiednio sformatowany zestaw, który może być efektywnie wykorzystany przez algorytmy uczenia maszynowego. Prawidłowo przygotowane dane prowadzą do bardziej dokładnych, skutecznych i wiarygodnych modeli predykcyjnych.

In [2]:
import pandas as pd
import glob
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import manipulating_data as md
from scipy.stats import iqr
import numpy as np
from dateutil import parser
from sklearn.linear_model import LinearRegression

# Zadanie 1

Celem tej instrukcji jest stworzenie klasy DataSet, która służy do reprezentowania danych tabelarycznych, użytecznych w modelowaniu predykcyjnym.

W ciele klasy DataSet, zdefiniuj metodę __init__, która inicjalizuje nową instancję klasy. Ta metoda powinna przyjmować argumenty path oraz data, z domyślnymi wartościami None. W ciele tej metody użyj instrukcji warunkowych, aby obsłużyć różne przypadki przekazania argumentów:

- Jeżeli podano tylko data, zapisz data jako atrybut instancji.
- Jeżeli podano tylko path, zapisz path jako atrybut instancji i wczytaj dane do atrybutu data za pomocą pd.read_csv(path).
- Jeżeli podano oba, wypisz wartość path oraz typ data i zgłoś wyjątek ValueError.
- W przypadku gdy nie podano żadnego z argumentów wypisz "Musimy skądś wziąć dane!" i zgłoś ValueError z komunikatem "Either path or data should be given." (lub podobnym).

Pamiętaj, jakie domyślne wartości mają argumenty konstruktora i że jeśli nie podamy któregoś argumentu, to będzie miał domyślną wartość.
To pomoże Ci ustalić, czy podano jeden argument, czy obydwa itd.

In [3]:
class DataSet() :

    """
    A class used to represents tabular data.
    which can be used in down-stream predictive modelling.

    Attributes
    ----------
    path : str
        The path to the data file.
    data : pandas DataFrame.
        The data loaded from the given path.

    """
    # konstruktor musi mieć taką nazwę jak zawsze
    def __init__(self, path=None, data=None):

        """
        Loads the data from the given path
        and stores path and data as instance attributes.
        The data attribute is of type pandas DataFrame.

        Parameters
        ----------
        path  : str, optional. Defaults to None.
            The path to the data file.
        data : pandas DataFrame, optional. Defaults to None.
            The data already in DataFrame form.
        """
        if path is not None and data is not None:
            print("Podano oba argumenty!")
            print(f"path: {path}")
            print(f"type(data): {type(data)}")
            raise ValueError("Provide either path or data, not both.")

        elif data is not None:
            self.data = data
            self.path = None

        elif path is not None:
            self.path = path
            try:
                self.data = pd.read_csv(path)
            except Exception as e:
                raise ValueError(f"Failed to read CSV file at {path}: {e}")

        else:
            print("Musimy skądś wziąć dane!")
            raise ValueError("Either 'path' or 'data' must be provided.")

# Zadanie 1.5 - zadanie na 5 dla ambitnych

### Rozszerzenie klasy CategoricalVariable
Cel:
Zadaniem jest stworzenie klasy NewCategoricalVariable, która dziedziczy po klasie CategoricalVariable i dodaje nową metodę one_hot_encode, która będzie realizować kodowanie one-hot dla zmiennej kategorycznej.

Ogólnie chodzi o to, że możemy mieć zmienną taką jak kolor, a modele machine learning / statystyczne rozumieją tylko liczby. Więc jak mamy zmienną kolor, o wartościach zielony i niebieski, to tworzymy 2 nowe zmienne, z których jedna odpowiada na pytanie "Czy kolor jest niebieski?" i przyjmuje wartość 1 jeśli tak i 0 jeśli nie, i drugą zmienną, która odpowiada na pytanie "Czy kolor jest zielony?" i również przyjmuje wartość 1, jeśli tak, i 0, jeśli nie.

### Rozszerzenie klasy CategoricalVariable
Cel:
Zadaniem jest stworzenie klasy NewCategoricalVariable, która dziedziczy po klasie CategoricalVariable i dodaje nową metodę one_hot_encode, która będzie realizować kodowanie one-hot dla zmiennej kategorycznej.

In [5]:
class CategoricalVariable() :

    """
    A class used to represent a categorical variable from some dataset.

    Attributes
    ----------
    column : pandas Series.
        A column from a pandas DataFrame or a standalone column.

    Methods
    -------
    ordinal_encode(column: pd.Series, show_mapping=False) -> pd.Series
        Encodes the given column using ordinal encoding.

    """


    def __init__(self, column : pd.Series) :
        self.column = column



    @staticmethod
    def ordinal_encode(column: pd.Series, show_mapping=False) -> pd.Series:
        encoder = OrdinalEncoder()
        encoder_fitted = encoder.fit(pd.DataFrame(column))
        encoded_data = encoder_fitted.transform(pd.DataFrame(column))
        inverse_transformation = encoder_fitted.inverse_transform(encoded_data)

        if show_mapping:
            values_mapping = { e.tolist()[0] : t.tolist() for t, e in\
                                        zip(encoded_data, inverse_transformation) }
            return values_mapping

        return pd.Series(encoded_data.flatten(), index=column.index, name=column.name)



    def encode_data(self, method, show_mapping=False) -> pd.DataFrame:
        if method == 'ordinal':
            encoded_df = CategoricalVariable.ordinal_encode(self.column, show_mapping=show_mapping)
        elif method == 'one_hot':
            encoded_df = CategoricalVariable.one_hot_encode(self.column, show_mapping=show_mapping)
        else:
            raise ValueError(f"Encoding method {method} not recognized.")
        return encoded_df


class NewCategoricalVariable(CategoricalVariable):
    @staticmethod
    def one_hot_encode(column: pd.Series) -> pd.DataFrame:
        # One-hot encoding z pandas
        frame = pd.get_dummies(column, prefix=column.name)
        return frame



In [6]:
# Przykład użycia klasy NewCategoricalVariable
data = pd.Series(['red', 'blue', 'red', 'green'])
cat = NewCategoricalVariable(data)
encoded = cat.one_hot_encode(data)
print(encoded)

    blue  green    red
0  False  False   True
1   True  False  False
2  False  False   True
3  False   True  False


- a) Zdefiniuj nową klasę NewCategoricalVariable, która dziedziczy po CategoricalVariable. To oznacza, że przejmie ona wszystkie atrybuty i metody klasy nadrzędnej.

- b) Napisz metodę one_hot_encode wewnątrz klasy NewCategoricalVariable. Ta metoda powinna przyjmować jedną kolumnę typu pd.Series tak jak metoda ordinal_encode w powyższej klasie CategoricalVariable. Nie musisz podawać żadnych innych argumentów. Metoda powinna zwracać DataFrame po kodowaniu one-hot.

Twoja nowa metoda w klasie NewCategoricalVariable ma wyglądać jakoś tak :  

In [ ]:
@staticmethod
def one_hot_encode(column : pd.Series) -> pd.DataFrame:

    # wewnątrz NewCategoricalVariable zdefiniuj tą metodę, a w środku
    # napisz kod który zamieni każdą unikalną wartość
    # zmiennej kategorialnej (takiej jak kolor czy nazwa miasta) na nową kolumnę
    # o wartościach 0 i 1 a następnie połączy te kolumny w jedną pd.DataFrame
    # o nazwie frame

    """
    Encodes the given column using one-hot encoding.

    Parameters
    ----------
    column : pd.Series
        The column to be encoded.

    Returns
    -------
    pd.DataFrame
        The one-hot encoded column, which becomes a DataFrame (table of columns).

    Example
    -------
    input :
        city
        sj
        sj
        iq
        iq

    output :
        city_iq  city_sj
            0.0      1.0
            0.0      1.0
            1.0      0.0
            1.0      0.0
    """

    return #frame

Przeanalizuj dokładnie przykład z docstringa aby zrozumieć, co Twój kod ma zrobić z danymi i w jakiej formie zwrócić wynik.

# Zadanie 2

### Instrukcja do napisania 2 metod wewnątrz klasy DateVariable

Klasa DateVariable służy do reprezentowania zmiennej daty z jakiegoś zbioru danych. Będziecie tworzyć dwie metody dla tej klasy: parse_string_date i encode_as_number. Oto jak to zrobić:
1. Metoda parse_string_date(string_date: str) -> pd.Timestamp:
Ta statyczna metoda ma za zadanie przekształcić datę w formacie ciągu znaków na obiekt pd.Timestamp (znacznik czasu pandas). Oto jakie kroki należy wykonać:

- a) Użyj metody pd.to_datetime() z biblioteki pandas, aby przekonwertować string_date na obiekt Timestamp.

- b) Zamień przekonwertowaną datę na standardowy format daty, np. 'YYYY-MM-DD', używając metody strftime('%Y-%m-%d').

- c) Zwróć przekształconą datę jako obiekt Timestamp.

Przykład szablonu metody:

In [7]:
class DateVariable():
    def __init__(self, column: pd.Series):
        self.column = column

    @staticmethod
    def parse_string_date(string_date: str) -> pd.Timestamp:
        parsed = pd.to_datetime(string_date)
        # Jeśli chcesz ustandaryzować format, możesz zwrócić to jako string:
        # return parsed.strftime('%Y-%m-%d')
        return parsed

    def encode_as_number(self) -> pd.Series:
        # Konwersja do datetime jeśli nie jest
        if self.column.dtype != 'datetime64[ns]':
            self.column = self.column.apply(DateVariable.parse_string_date)

        year = self.column.dt.year
        month = self.column.dt.month
        day = self.column.dt.day

        encoded = year * 365 + (month - 1) * 30 + day
        return encoded


In [8]:
# Przykład użycia klasy DateVariable
dates = pd.Series(['2022-01-01', '2022-01-02', '2023-02-10'])
date_var = DateVariable(dates)
encoded_dates = date_var.encode_as_number()
print(encoded_dates)

0    738031
1    738032
2    738435
dtype: int32


2. Metoda encode_as_number() -> pd.Series:
Ta metoda ma za zadanie zakodować datę jako liczbę, gdzie każdy dzień jest reprezentowany przez unikalną liczbę. Realizacja tej metody powinna obejmować następujące kroki:

- a) Sprawdź, czy typ danych w self.column to 'datetime64[ns]'. Jeśli nie, użyj metody apply z biblioteki pandas na self.column, aby zamienić każdą datę (zakładając, że są one w formacie ciągu znaków) na format Timestamp, używając wcześniej napisanej metody statycznej parse_string_date.

- b) Po upewnieniu się, że wszystkie daty są w formacie Timestamp, wyodrębnij z nich rok, miesiąc i dzień. Możesz to zrobić korzystając z atrybutów .dt.year, .dt.month i .dt.day obiektu Series pandas.

- c) Zakoduj daty jako liczby, przypisując każdemu roku wartość 365 (liczba dni w roku), każdemu miesiącu wartość 30 (przybliżona liczba dni w miesiącu) i dodając do tego liczbę dni. Wzór to rok * 365 + (miesiąc - 1) * 30 + dzień.

Przykład szablonu metody:

In [ ]:
# tu nic nie robisz, masz to zrobić wewnątrz klasy DateVariable, która jest nieco niżej

def encode_as_number(self) -> pd.Series:
    # Tu wstaw logikę kodowania daty jako liczby.
    return # zakodowana jako liczba pd.Series


W tej klasie musisz napisać 2 metody opisane w poleceniu :

In [ ]:
class DateVariable() :

    """
    A class used to represent a date variable from some dataset.

    Attributes
    ----------
    column : pandas Series.
        A column from a pandas DataFrame or a standalone column.

    Static Methods
    -------
    parse_string_date(string_date : str) -> pd.Timestamp
        Parses a string date into a pandas Timestamp object.

    Methods
    -------
    encode_as_number() -> pd.Series
        Encodes the date as a number, where each day is represented by a unique number.
    """

    def __init__(self, column : pd.Series) :
        self.column = column

    @staticmethod
    def parse_string_date(string_date : str) :

        pass


    def encode_as_number(self) :

        pass


 Na koniec zobacz czy metoda encode_as_number poprawnie koduje daty jako liczby.

# Zadanie 3

Klasa NumericVariable jest przeznaczona do reprezentowania zmiennej numerycznej z jakiegoś zbioru danych. Będziecie tworzyć konstruktor oraz metodę detect_outlier_iqr. Oto jak to zrobić:
1. Konstruktor __init__(self, column: pd.Series):
Konstruktor ma za zadanie inicjalizować instancję klasy, przyjmując jedną zmienną: kolumnę danych w formacie pd.Series. Oto jakie kroki należy wykonać:

- a) W definicji konstruktora określ, że przyjmuje on jeden parametr column, który jest obiektem typu pd.Series.

- b) W ciele konstruktora przypisz przekazany argument column do atrybutu instancji o tej samej nazwie column.

2. Metoda detect_outlier_iqr(self) -> list:
Ta metoda ma za zadanie wykrywać obserwacje odstające (outliers) w przekazanej kolumnie numerycznej, korzystając z metody zakresu międzykwartylowego (IQR). Realizacja tej metody powinna obejmować następujące kroki:

- a) Oblicz pierwszy (Q1) i trzeci (Q3) kwartyl danych za pomocą funkcji np.percentile z biblioteki numpy.

- b) Oblicz zakres międzykwartylowy (IQR) jako różnicę między Q3 a Q1.

- c) Określ wartości graniczne dla wykrywania odstających obserwacji: 1.5 * IQR powyżej Q3 i 1.5 * IQR poniżej Q1.

- d) Użyj tych granic do identyfikacji indeksów obserwacji odstających, które są mniejsze niż dolna granica lub większe niż górna granica.

- e) Zwróć listę indeksów tych obserwacji odstających.

Przykład szablonu metody:

In [ ]:
# Tu nic nie robisz, masz to zrobić wewnątrz klasy NumericVariable, która jest niżej

def detect_outlier_iqr(self) -> list:
    # Tu wstaw logikę wykrywania obserwacji odstających.
    return # lista indeksów obserwacji odstających


*** Początek wskazówki do podpunktu d) ***

Pamiętaj, że Twój atrybut instancji klasy NumericVariable jest typu pd.Series.

In [ ]:
# przykładowy obiekt typu pd.Series
seria = pd.Series([1, 2 ,3])
seria

0    1
1    2
2    3
dtype: int64

In [ ]:
# subsetting obiektu pd.Series
# wyświetlamy elementy mniejsze od 3
seria[seria < 3]

0    1
1    2
dtype: int64

In [ ]:
# w zadaniu będziemy bazować na indeksach
# poniżej przykład jak znależć indeksy danych mniejszych od 3 i dostać je jako listę
seria[seria < 3].index.tolist()

[0, 1]

W d) musisz zrobić to samo dla górnej i dolnej wartości granicznej a następnie połączyć te 2 listy.

\\\ Koniec wskazówki do podpunktu d) \\\

In [10]:
class NumericVariable:
    """
    A class used to represent a numeric variable from some dataset.

    Attributes
    ----------
    column : pandas Series
        A column from a pandas DataFrame or a standalone column.

    Methods
    -------
    detect_outlier_iqr() -> list
        Detects outliers using the interquartile range method.
    """

    def __init__(self, column: pd.Series):
        """
        Initializes the NumericVariable instance.

        Parameters
        ----------
        column : pd.Series
            A numeric column.
        """
        self.column = column

    def detect_outlier_iqr(self) -> list:
        """
        Detects outliers using the interquartile range (IQR) method.
        Returns a list of indices of the outliers in the given column.

        Returns
        -------
        list
            A list of indices of the outliers in the given column.
        """
        q1 = np.percentile(self.column, 25)
        q3 = np.percentile(self.column, 75)
        iqr = q3 - q1

        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        outliers_indices = self.column[
            (self.column < lower_bound) | (self.column > upper_bound)
        ].index.tolist()

        return outliers_indices

Kiedy skończysz uruchom poniższy kod :

In [11]:
NumericVariable(pd.Series( [ 1, 2, 3, 10 ** 7 ] ) ).detect_outlier_iqr()

[3]

Powinien zwrócić 3, czyli indeks liczby nie pasującej wielkością do reszty - 10 ^ 7.

In [ ]:
class CategoricalData(DataSet) :
    """
    A class used to represent a set of categorical variables from some dataset.
    Inherits from the DataSet class.

    Attributes
    ----------
    path : str, optional. Defaults to None.
        The path to the data file.
    data : pandas DataFrame, optional. Defaults to None.
        The data already in DataFrame form.
    max_uniq_vals : int, optional. Defaults to 10.
        The maximum number of unique values a column can have.
        If a column has more unique values than this, it will not be encoded.
        It is useful for avoiding computational overhead when one-hot encoding.

    Methods
    -------
    encode_data(method, show_mapping=False) -> pd.DataFrame
        Encodes the categorical data using the given method.

    """

    def __init__(self, path = None, data = None, max_uniq_vals=10) :
        super().__init__(path, data)
        self.cat_data = self.data.select_dtypes(include='object')
        self.unique_values = self.cat_data.nunique()
        self.cols_to_encode = self.unique_values[self.unique_values <= max_uniq_vals].index.tolist()
        self.cat_data = self.data[self.cols_to_encode]

    def encode_data(self, method, show_mapping=False) -> pd.DataFrame:

        encoded_data = {}

        for column in self.cat_data.columns:
            categorical_col = CategoricalVariable(self.cat_data[column])
            encoded_data[column] = categorical_col.encode_data(method, show_mapping)

        df = pd.DataFrame()

        for k, v in encoded_data.items():
            if method == 'ordinal':
                df[k] = v
            elif method == 'one_hot':
                df = pd.concat([df, v], axis=1)
            else:
                raise ValueError(f"Encoding method {method} not recognized.")

        return df

In [ ]:
class NumericData(DataSet) :

    """
    A class used to represent numeric data from some dataset.
    Inherits from the DataSet class (which can consist of both numeric and categorical data).

    Attributes
    ----------
    path : str, optional. Defaults to None.
        The path to the data file.
    data : pandas DataFrame, optional. Defaults to None.
        The data already in DataFrame form.

    Methods
    -------
    detect_outliers(method='iqr', by_column=False) -> list or dict
        Detects outliers using the given method.
        Defaults to the interquartile range method.
    """

    def __init__(self, path = None, data=None) :

        super().__init__(path, data)
        self.num_data = self.data.select_dtypes(include='number')

    def detect_outliers(self, method = 'iqr', by_column = False) :

        """
        Detects outliers using the given method.
        By default uses interquartile range method.
        Under the hood, it applies NumericVariable.detect_outlier_iqr() to each numeric column,
        and returns combined data as list / dictionary.

        Parameters
        ----------
        method : str, optional. Defaults to 'iqr'.
            The method to use for outlier detection.

        by_column : bool, optional. Defaults to False.
            If True, returns a dictionary of outliers for each column.
            If False, returns a list of indices of the outliers in the dataset (which is a set of all columns).

        Returns
        -------
        list or dict
            A list of indices of the outliers in the dataset (if by_column is False).
            A dictionary of outliers for each column (if by_column is True).
        """

        outliers = {}

        for c in self.num_data.columns :

            numeric_col = NumericVariable(self.data[c])

            if method == 'iqr' :
                indices_outliers_iterab = numeric_col.detect_outlier_iqr()

                if indices_outliers_iterab != [] :
                    outliers[c] = indices_outliers_iterab


            elif method == 'z_score' :
                pass
            else :
                raise ValueError(f"Outlier detection method {method} not recognized.")

        if by_column :
            return outliers

        outlier_indices = []

        for v in outliers.values() :
            outlier_indices += v

        return outlier_indices

# Zadanie 4

Przeanalizuj proszę klasę DataImputation i napisz metody mean_imputation i median_imputation w sposób podobny do tego, w jaki została napisana metoda mode_imputation. Zwróć uwagę, że w klasie brakuje konstruktora, dopisz odpowiedni konstruktor - musi mieć taki argument, żeby metoda mode_imputation i bliżniacze metody które napiszesz działały.

In [12]:
class DataImputation:
    """
    A class used to represent data imputation methods - methods used to fill in missing values in a dataset.

    Attributes
    ----------
    data : pd.DataFrame
        The dataset on which imputations will be performed.

    Methods
    -------
    mode_imputation(column : pd.Series) -> pd.Series
        Imputes missing values using the mode of the given column.

    mean_imputation(column : pd.Series) -> pd.Series
        Imputes missing values using the mean of the given column.

    median_imputation(column : pd.Series) -> pd.Series
        Imputes missing values using the median of the given column.
    """

    def __init__(self, data: pd.DataFrame):
        self.data = data

    def mode_imputation(self, column: pd.Series) -> pd.Series:
        mode = column.mode()[0]
        return column.fillna(mode)

    def mean_imputation(self, column: pd.Series) -> pd.Series:
        mean = column.mean()
        return column.fillna(mean)

    def median_imputation(self, column: pd.Series) -> pd.Series:
        median = column.median()
        return column.fillna(median)


In [13]:
class PreparingDataset(CategoricalData, NumericData) :

    """
    A class used to describe how to prepare data for predictive modelling.
    Inherits from both CategoricalData and NumericData classes.

    Attributes
    ----------
    path : str, optional. Defaults to None.
        The path to the data file.
    data : pandas DataFrame, optional. Defaults to None.
        The data already in DataFrame form.
    date_col_name : str, optional. Defaults to None.
        The name of the column that contains date data.

    Methods
    -------
    prepare_categoricl_data(method='one_hot', impute_missing=False) -> pd.DataFrame
        Prepares the categorical data for predictive modelling, meaning it
        fills in the missing values with the mode (most common value)
        and encodes values such as 'green' as numbers."""

    def __init__(self, path = None, data = None, date_col_name = None) :

        CategoricalData.__init__(self, path = path, data = data)
        NumericData.__init__(self, path = path, data = data)

        if date_col_name is not None :
            self.date_data = self.data[date_col_name]

    def prepare_categoricl_data(self, method = 'one_hot', impute_missing = False) :

        if impute_missing :
            for c in self.cat_data.columns :
                most_common = self.cat_data[c].mode()[0]
                self.cat_data[c] = self.cat_data[c].fillna(most_common)

        return self.encode_data(method = method)

    def prepare_numeric_data(self, method = 'iqr', remove_outliers = False, impute_missing = False) :

        outliers = self.detect_outliers(method)
        outliers_by_column = self.detect_outliers(method, by_column=True)

        if impute_missing :
            for c in self.num_data.columns :
                if c in outliers_by_column.keys() :
                    median = self.num_data[c].median()
                    self.num_data[c] = self.num_data[c].fillna(median)
                else :
                    mean = self.num_data[c].mean()
                    self.num_data[c] = self.num_data[c].fillna(mean)

        if remove_outliers :
            indices_keep = [i for i in self.num_data.index if i not in outliers]
            self.num_data = self.num_data.iloc[indices_keep]

            return self.num_data

        else :

            return self.num_data

    def prepare_date_data(self) :

        return DateVariable(self.date_data).encode_as_number()





NameError: name 'CategoricalData' is not defined

In [ ]:
class CleanDataset(PreparingDataset) :

    """
    A class used to represent a clean dataset which is ready for predictive modelling.

    Attributes
    ----------
    path : str, optional. Defaults to None.
        The path to the data file.
    data : pandas DataFrame, optional. Defaults to None.
        The data already in DataFrame form.
    date_col_name : str, optional. Defaults to None.
        The name of the column that contains date data.

    Methods
    -------
    get_data(encoding_method='one_hot', outlier_method='iqr', remove_outliers=True, impute_missing=False) -> pd.DataFrame
        Returns the clean dataset ready for predictive modelling.

    """

    def __init__(self, path = None, data = None, date_col_name = None) :
        super().__init__(path = path, data = data, date_col_name = date_col_name)
        self.date_col_name = date_col_name

    def get_data(self, encoding_method = 'one_hot', outlier_method = 'iqr', remove_outliers = True, impute_missing = False) :

        categorical = self.prepare_categoricl_data(method = encoding_method, impute_missing = impute_missing)
        numeric = self.prepare_numeric_data(method=outlier_method, remove_outliers = remove_outliers, impute_missing=impute_missing)

        data_parts = [categorical, numeric]

        if self.date_col_name is not None :
            date_calendar = self.prepare_date_data()
            data_parts.append(date_calendar)

        return pd.concat(data_parts, axis=1)

In [ ]:
path = glob.glob('**/*dengue_features_train.csv', recursive=True) [0]

In [ ]:
deng_train = pd.read_csv(path)

## Przykład danych wejściowych

In [ ]:
deng_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [ ]:
deng_train[['city', 'week_start_date']].head()

,city,week_start_date
0,sj,1990-04-30
1,sj,1990-05-07
2,sj,1990-05-14
3,sj,1990-05-21
4,sj,1990-05-28


In [ ]:
df = CleanDataset(data = deng_train, date_col_name = 'week_start_date').get_data(impute_missing=True, remove_outliers=True, encoding_method='ordinal')

C:\Users\Predator\AppData\Local\Temp\ipykernel_21896\2005783920.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.cat_data[c] = self.cat_data[c].fillna(most_common)
C:\Users\Predator\AppData\Local\Temp\ipykernel_21896\2005783920.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.num_data[c] = self.num_data[c].fillna(mean)


## Przykład danych wyjściowych

In [ ]:
df.head()

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,week_start_date
0,1.0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,726470
1,1.0,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,726477
2,1.0,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,726484
3,1.0,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,726491
4,1.0,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,726498


In [ ]:
df[['city', 'week_start_date']].head()

,city,week_start_date
0,1.0,726470
1,1.0,726477
2,1.0,726484
3,1.0,726491
4,1.0,726498


# Zadanie 5

Podziel proszę program na 3 moduły :
- main,
- utils, z klasami, które nie dziedziczą i nie są dziedziczone - DataImputation, DateVariable,
- z resztą klas.

### pliki main, utils i data_preparation